<a href="https://colab.research.google.com/github/slagathorr/nyc-mta-explore/blob/main/mta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started

We're going to do some setup:


*   Install Python libraries.
*   (input required) Set up some environment variables.
*   Set up some variables.

In [1]:
# Install Google's GTFS library. This may ask Colab to restart the runtime.
!pip3 install --upgrade "gtfs-realtime-bindings==1.0.0"

# Test the GTFS module. This should output nothing if successful.
!python3 -m google.transit.gtfs_realtime_pb2

from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToJson

import duckdb # We're going to need some ducks.

import os
import requests
import pytz
import datetime

In [2]:
# Enter your MTA API key here.
# If you don't have one, you can get one here: https://api.mta.info/#/landing
API_KEY="13luoawJzW7wTQlVyMc282wToiVyv9iG31OYViSV"

In [3]:
# Set static station data variables.
# According to the MTA data page, there is a hosted CSV for station data.
# http://web.mta.info/developers/developer-data-terms.html#data
station_csv_url = "https://atisdata.s3.amazonaws.com/Station/Stations.csv"
station_s3_region_curl = !curl -sI https://atisdata.s3.amazonaws.com | grep bucket-region
station_s3_region = station_s3_region_curl[0].split()[1]

# Creating a dictionary of MTA Subway API endpoints.
url_subway = {
    "ace": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-ace",
    "g": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-g",
    "bdfm": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-bdfm",
    "jz": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-jz",
    "nqrw": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-nqrw",
    "l": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-l",
    "1234567": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs",
    "sir": "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-si"}

# Load Static Station Data
We have some static data around the subway stop locations that we would like to load. This is hosted in a public S3 bucket as a CSV, and we're going to connect to DuckDB and load this into a table. This will show ten rows of that so you can get a sense of what the data looks like.

In [4]:
conn = duckdb.connect()
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")
conn.execute("SET s3_region='"+station_s3_region+"'")

conn.execute("CREATE OR REPLACE TABLE stations AS SELECT * FROM '"+station_csv_url+"'")
conn.execute("SELECT * FROM stations LIMIT 10").fetchdf()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Direction Notes,ADA NB,ADA SB,Capital Outage NB,Capital Outage SB
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN,NaN,NaN,NaN,NaN
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN,NaN,NaN,NaN,NaN
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN,NaN,NaN,NaN,NaN
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN,NaN,NaN,NaN,NaN
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN,NaN,NaN,NaN,NaN
5,6,6,R08,BMT,Astoria,39 Av-Dutch Kills,Q,N W,Elevated,40.752882,-73.932755,Astoria - Ditmars Blvd,Manhattan,0,NaN,NaN,NaN,NaN,NaN
6,7,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn,0,NaN,NaN,NaN,NaN,NaN
7,8,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn,0,NaN,NaN,NaN,NaN,NaN
8,9,9,R14,BMT,Broadway - Brighton,57 St-7 Av,M,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn,1,NaN,NaN,NaN,NaN,NaN
9,10,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn,2,Uptown & Queens,1.0,0.0,NaN,NaN


# Looking at One Subway Line
We will now try pulling data from one of lines to show how this works. We will use the G line since it's not a mixed line and will be easy to show as an example.

## Construct the API request.
Here we simply have to make an HTTP call to the endpoint, and pass your API key in the header as an `x-api-key`. Here we will print the response header to verify that it worked.

In [6]:
headers = {'x-api-key': API_KEY}

response = requests.get(url_subway['g'], headers=headers, allow_redirects=True)

print(response.headers)

# Helper function to reformat the API header time format into a format that DuckDB will take.
def reformat_time(input_time):
  datetime_input_time = datetime.datetime.strptime(input_time, '%a, %d %b %Y %H:%M:%S GMT')
  return datetime_input_time.strftime('%Y-%m-%d %H:%M:%S') + "+00"

print(reformat_time(response.headers["Date"]))

{'Content-Type': 'text/plain', 'Content-Length': '16223', 'Connection': 'keep-alive', 'Date': 'Wed, 12 Jul 2023 14:28:53 GMT', 'x-amzn-RequestId': '123e7d79-ef59-4a29-b0c8-10fc04a1f1c5', 'Access-Control-Allow-Origin': '*', 'x-amz-apigw-id': 'H9HJ1GN3IAMF7QA=', 'X-Amzn-Trace-Id': 'Root=1-64aeb8a5-36720dc47e55654e13135605', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 37e37f3d71c99a5a71f8e766a6f4b9ee.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'LAX50-C1', 'X-Amz-Cf-Id': 'QI60sVsk66NkCdsIxGnfQj3_ZW3rUEH6WRnPgc4zRXqp3BsmoLHHig=='}
2023-07-12 14:28:53+00


## Parse and Load the Response
We will now use Google's GTFS Python Library to parse the response. The `trip_update` objects are the ones that we will be looking for and loading (you can find more details on the entities [here](https://doodles.patrickweaver.net/working-with-nyc-mta-realtime-subway-data/slides/4/).

We will also be creating a table called `landing_subway_trip_updates` to load this into. This will be cleaned up later.

In [ ]:
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

conn.execute("CREATE OR REPLACE TABLE landing_subway_trip_updates (api_request_time TIMESTAMP WITH TIME ZONE, subway_updates JSON)")

# Iterate through the responses.
# Print the first five repsponses to inspect.
i=0
for entity in feed.entity:
  if entity.HasField('trip_update'):
    update_json = MessageToJson(entity)
    conn.execute("INSERT INTO landing_subway_trip_updates VALUES ('" + reformat_time(response.headers["Date"]) + "'::TIMESTAMPTZ, ?)", [update_json.replace('\n','')])
    if i < 5: print(update_json)
    i = i+1
print("inserted {} rows".format(i))

## Taking a Look at the Data
Let's now see what the data looks like. Querying the table gives us a glimpse into the JSON object's structure.

In [28]:
conn.execute("""
  SELECT
    subway_updates AS subway_updates_original_message,
    api_request_time,
    api_request_time AT TIME ZONE 'US/Eastern' as api_request_time_eastern,
    subway_updates->'$.tripUpdate.trip.routeId' as subway_line,
    subway_updates->'$.tripUpdate.trip.tripId' as trip_id,
    subway_updates->'$.tripUpdate.trip.startTime' as start_time,
    subway_updates->'$.tripUpdate.trip.startDate' as start_date,
    subway_updates->'$.tripUpdate.stopTimeUpdate' as stopTimeUpdate
  FROM landing_subway_trip_updates
""").df()

,subway_updates_original_message,api_request_time,api_request_time_eastern,subway_line,trip_id,start_time,start_date,stopTimeUpdate
0,"{ ""id"": ""000001G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""058250_G..N""","""09:42:30""","""20230712""",NaN
1,"{ ""id"": ""000003G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""058600_G..S""","""09:46:00""","""20230712""","[{""arrival"":{""time"":""1689172113""},""departure"":..."
2,"{ ""id"": ""000005G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""059580_G..N""","""09:55:48""","""20230712""","[{""arrival"":{""time"":""1689172118""},""departure"":..."
3,"{ ""id"": ""000007G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""059563_G..S""","""09:55:38""","""20230712""","[{""arrival"":{""time"":""1689172118""},""departure"":..."
4,"{ ""id"": ""000009G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""060600_G..N""","""10:06:00""","""20230712""","[{""arrival"":{""time"":""1689172113""},""departure"":..."
5,"{ ""id"": ""000011G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""060583_G..S""","""10:05:50""","""20230712""","[{""arrival"":{""time"":""1689172118""},""departure"":..."
6,"{ ""id"": ""000013G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""061451_G..N""","""10:14:31""","""20230712""","[{""arrival"":{""time"":""1689172118""},""departure"":..."
7,"{ ""id"": ""000015G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""062600_G..S""","""10:26:00""","""20230712""","[{""arrival"":{""time"":""1689172183""},""departure"":..."
8,"{ ""id"": ""000017G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""061481_G..S""","""10:14:49""","""20230712""","[{""arrival"":{""time"":""1689172118""},""departure"":..."
9,"{ ""id"": ""000019G"", ""tripUpdate"": { ""trip""...",2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,"""G""","""062650_G..N""","""10:26:30""","""20230712""","[{""arrival"":{""time"":""1689172110""},""departure"":..."


From the above, we can see the general structure of these entries.


*   There is an `id` for each entry.
*   Each entry has a `tripUpdate` which has details of each train.
  *   At this level we have a `trip` entry that details the `tripId` of each train, the `startTime` of the train, the `startDate`, and the `routeId` which is the train line designation.
  *   There is then the `stopTimeUpdate` element which is an array of stop arrival and departure times for that train.



## Transforming the Data

To make the analysis a little bit easier, we're going to create a new table which extracts some of the data out into their own columns. We are also going to do some restructuring.


*   
*   List item



In [85]:
#throwaway

conn.sql("""
  WITH stg_subway_trip_updates_1 AS(
  SELECT
    subway_updates AS subway_updates_original_message,
    api_request_time,
    api_request_time AT TIME ZONE 'US/Eastern' as api_request_time_eastern,
    subway_updates->>'$.tripUpdate.trip.routeId' as subway_line,
    subway_updates->>'$.tripUpdate.trip.tripId' as trip_id,
    strptime(subway_updates->>'$.tripUpdate.trip.startDate','%Y%m%d')::DATE as start_date,
    (subway_updates->>'$.tripUpdate.trip.startTime')::TIME as start_time,
    (subway_updates->'$.tripUpdate.stopTimeUpdate')::STRUCT(arrival STRUCT(time BIGINT), departure STRUCT(time BIGINT), stopId VARCHAR)[] as restructured,
    IF((subway_updates->'$.tripUpdate.stopTimeUpdate') IS NULL, [], subway_updates->'$.tripUpdate.stopTimeUpdate') as stopTimeUpdate
  FROM landing_subway_trip_updates),

  stg_subway_trip_updates_2 AS(
  SELECT
    trip_id,
    UNNEST(stopTimeUpdate::STRUCT(arrival STRUCT(time BIGINT), departure STRUCT(time BIGINT), stopId VARCHAR)[]) as restructured
  FROM stg_subway_trip_updates_1
  UNION ALL
  SELECT
    trip_id,
    NULL as restructured
  FROM stg_subway_trip_updates_1
  WHERE restructured IS NULL  )

SELECT
    *
  FROM stg_subway_trip_updates_2
  WHERE trip_id = '058250_G..N'

""").show()

┌─────────────┬────────────────────────────────────────────────────────────────────────────────────────┐
│   trip_id   │                                      restructured                                      │
│   varchar   │ struct(arrival struct("time" bigint), departure struct("time" bigint), stopid varchar) │
├─────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│ 058250_G..N │ NULL                                                                                   │
└─────────────┴────────────────────────────────────────────────────────────────────────────────────────┘



In [91]:
# Create a new table
# stg_subway_trip_updates_1 -> Extract out some columns, and get the stopTimeUpdate list out.
# stg_subway_trip_updates_2 -> take the stopTimeUpdate list, and unnest the thing.
# stg_array_restructured -> Reformat the timestamps, join it with the station static data, and array_agg it back up.
# SELECT join the two by the trip_id, which we know is a unique identifier.
conn.execute("""
  WITH stg_subway_trip_updates_1 AS(
  SELECT
    subway_updates AS subway_updates_original_message,
    api_request_time,
    api_request_time AT TIME ZONE 'US/Eastern' as api_request_time_eastern,
    subway_updates->>'$.tripUpdate.trip.routeId' as subway_line,
    subway_updates->>'$.tripUpdate.trip.tripId' as trip_id,
    strptime(subway_updates->>'$.tripUpdate.trip.startDate','%Y%m%d')::DATE as start_date,
    (subway_updates->>'$.tripUpdate.trip.startTime')::TIME as start_time,
    IF((subway_updates->'$.tripUpdate.stopTimeUpdate') IS NULL, [], subway_updates->'$.tripUpdate.stopTimeUpdate') as stopTimeUpdate
  FROM landing_subway_trip_updates),

  stg_subway_trip_updates_2 AS(
  SELECT
    trip_id,
    UNNEST(stopTimeUpdate::STRUCT(arrival STRUCT(time BIGINT), departure STRUCT(time BIGINT), stopId VARCHAR)[]) as restructured
  FROM stg_subway_trip_updates_1
  UNION ALL
  SELECT
    trip_id,
    NULL as restructured
  FROM stg_subway_trip_updates_1
  WHERE restructured IS NULL),

  stg_array_restructured AS(
  SELECT
    trip_id,
    ARRAY_AGG({ 'arrival_time': to_timestamp(restructured.arrival.time),
      'departure_time': to_timestamp(restructured.departure.time),
      'stop_id': restructured.stopId,
      'station_name': (SELECT \"Stop Name\" FROM stations where \"GTFS Stop ID\" =  restructured.stopId[:-1])})  as newStopTimeUpdate
  FROM stg_subway_trip_updates_2 GROUP BY trip_id)

  SELECT
    stg1.* EXCLUDE(stopTimeUpdate, subway_updates_original_message),
    stg2.newStopTimeUpdate AS train_stop_details,
    stg1.subway_updates_original_message
  FROM stg_subway_trip_updates_1 stg1, stg_array_restructured stg2
  WHERE stg1.trip_id = stg2.trip_id
""").df()

,api_request_time,api_request_time_eastern,subway_line,trip_id,start_date,start_time,train_stop_details,subway_updates_original_message
0,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,058600_G..S,2023-07-12,09:46:00,"[{'arrival_time': 2023-07-12 14:28:33, 'depart...","{ ""id"": ""000003G"", ""tripUpdate"": { ""trip""..."
1,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,059580_G..N,2023-07-12,09:55:48,"[{'arrival_time': 2023-07-12 14:28:38, 'depart...","{ ""id"": ""000005G"", ""tripUpdate"": { ""trip""..."
2,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,059563_G..S,2023-07-12,09:55:38,"[{'arrival_time': 2023-07-12 14:28:38, 'depart...","{ ""id"": ""000007G"", ""tripUpdate"": { ""trip""..."
3,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,060600_G..N,2023-07-12,10:06:00,"[{'arrival_time': 2023-07-12 14:28:33, 'depart...","{ ""id"": ""000009G"", ""tripUpdate"": { ""trip""..."
4,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,060583_G..S,2023-07-12,10:05:50,"[{'arrival_time': 2023-07-12 14:28:38, 'depart...","{ ""id"": ""000011G"", ""tripUpdate"": { ""trip""..."
5,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,061451_G..N,2023-07-12,10:14:31,"[{'arrival_time': 2023-07-12 14:28:38, 'depart...","{ ""id"": ""000013G"", ""tripUpdate"": { ""trip""..."
6,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,062600_G..S,2023-07-12,10:26:00,"[{'arrival_time': 2023-07-12 14:29:43, 'depart...","{ ""id"": ""000015G"", ""tripUpdate"": { ""trip""..."
7,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,061481_G..S,2023-07-12,10:14:49,"[{'arrival_time': 2023-07-12 14:28:38, 'depart...","{ ""id"": ""000017G"", ""tripUpdate"": { ""trip""..."
8,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,062650_G..N,2023-07-12,10:26:30,"[{'arrival_time': 2023-07-12 14:28:30, 'depart...","{ ""id"": ""000019G"", ""tripUpdate"": { ""trip""..."
9,2023-07-12 14:28:53+00:00,2023-07-12 10:28:53,G,063400_G..N,2023-07-12,10:34:00,"[{'arrival_time': 2023-07-12 14:34:00, 'depart...","{ ""id"": ""000021G"", ""tripUpdate"": { ""trip""..."


In [ ]:
conn.execute("DESCRIBE stg_subway_trip_updates").df()

,column_name,column_type,null,key,default,extra
0,input_time,TIMESTAMP WITH TIME ZONE,YES,NaN,NaN,NaN
1,"main.timezone('US/Eastern', input_time)",TIMESTAMP,YES,NaN,NaN,NaN
2,subway_line,VARCHAR,YES,NaN,NaN,NaN
3,trip_id,VARCHAR,YES,NaN,NaN,NaN
4,start_date,DATE,YES,NaN,NaN,NaN
5,start_time,TIME,YES,NaN,NaN,NaN
6,restructured,"STRUCT(arrival STRUCT(""time"" BIGINT), departur...",YES,NaN,NaN,NaN
7,stopTimeUpdate,JSON,YES,NaN,NaN,NaN
8,subway_updates,JSON,YES,NaN,NaN,NaN


In [ ]:
conn.execute("SELECT * FROM stg_subway_trip_updates order by trip_id asc LIMIT 5").df()

,input_time,"main.timezone('US/Eastern', input_time)",subway_line,trip_id,start_date,start_time,restructured,stopTimeUpdate,subway_updates
0,2023-07-11 19:05:27+00:00,2023-07-11 15:05:27,G,085416_G..N,2023-07-11,14:14:10,NaN,NaN,"{ ""id"": ""000001G"", ""tripUpdate"": { ""trip""..."
1,2023-07-11 19:05:27+00:00,2023-07-11 15:05:27,G,086603_G..S,2023-07-11,14:26:02,"[{'arrival': {'time': 1689102379}, 'departure'...","[{""arrival"":{""time"":""1689102379""},""departure"":...","{ ""id"": ""000003G"", ""tripUpdate"": { ""trip""..."
2,2023-07-11 19:05:27+00:00,2023-07-11 15:05:27,G,086700_G..N,2023-07-11,14:27:00,NaN,NaN,"{ ""id"": ""000005G"", ""tripUpdate"": { ""trip""..."
3,2023-07-11 19:05:27+00:00,2023-07-11 15:05:27,G,087278_G..N,2023-07-11,14:32:47,"[{'arrival': {'time': 1689102311}, 'departure'...","[{""arrival"":{""time"":""1689102311""},""departure"":...","{ ""id"": ""000007G"", ""tripUpdate"": { ""trip""..."
4,2023-07-11 19:05:27+00:00,2023-07-11 15:05:27,G,087628_G..S,2023-07-11,14:36:17,"[{'arrival': {'time': 1689102311}, 'departure'...","[{""arrival"":{""time"":""1689102311""},""departure"":...","{ ""id"": ""000009G"", ""tripUpdate"": { ""trip""..."


In [ ]:
#
conn.execute("""describe select trip_id, stopTimeUpdate,
  stopTimeUpdate::STRUCT(arrival STRUCT(time BIGINT), departure STRUCT(time BIGINT), stopId VARCHAR)[] as restructured
  from stg_subway_trip_updates order by trip_id asc """).df()

,column_name,column_type,null,key,default,extra
0,trip_id,VARCHAR,YES,NaN,NaN,NaN
1,stopTimeUpdate,JSON,YES,NaN,NaN,NaN
2,restructured,"STRUCT(arrival STRUCT(""time"" BIGINT), departur...",YES,NaN,NaN,NaN


In [ ]:
conn.execute("""
WITH temptable AS (
  SELECT
    trip_id,
    unnest(stopTimeUpdate::STRUCT(arrival STRUCT(time BIGINT), departure STRUCT(time BIGINT), stopId VARCHAR)[]) as restructured
  FROM stg_subway_trip_updates
  ORDER BY trip_id ASC)
SELECT
  trip_id,
  ARRAY_AGG({ 'arrival_time': to_timestamp(restructured.arrival.time)::TIMESTAMPTZ,
    'departure_time': to_timestamp(restructured.departure.time)::TIMESTAMPTZ,
    'stop_id': restructured.stopId,
    'station_name': (SELECT \"Stop Name\" FROM stations where \"GTFS Stop ID\" =  restructured.stopId[:-1])})  as newStopTimeUpdate
FROM temptable GROUP BY trip_id
""").df()

,trip_id,newStopTimeUpdate
0,086603_G..S,"[{'arrival_time': 2023-07-11 19:06:19, 'depart..."
1,087278_G..N,"[{'arrival_time': 2023-07-11 19:05:11, 'depart..."
2,087628_G..S,"[{'arrival_time': 2023-07-11 19:05:11, 'depart..."
3,088485_G..N,"[{'arrival_time': 2023-07-11 19:12:49, 'depart..."
4,088600_G..S,"[{'arrival_time': 2023-07-11 19:05:11, 'depart..."
5,089551_G..N,"[{'arrival_time': 2023-07-11 19:25:19, 'depart..."
6,089600_G..S,"[{'arrival_time': 2023-07-11 19:23:19, 'depart..."
7,090200_G..N,"[{'arrival_time': 2023-07-11 19:08:19, 'depart..."
8,090500_G..S,"[{'arrival_time': 2023-07-11 19:25:30, 'depart..."
9,090900_G..N,"[{'arrival_time': 2023-07-11 19:09:00, 'depart..."


In [ ]:
conn.execute("select to_timestamp(1689102379::BIGINT)::timestamptz").df()

,CAST(to_timestamp(CAST(1689102379 AS BIGINT)) AS TIMESTAMP WITH TIME ZONE)
0,2023-07-11 19:06:19+00:00
